In [7]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [14]:
ratings_df = pd.read_csv(r"C:\Users\Linzhiping\Jupyter\ml-latest-small\ratings.csv")
ratings_df.tail()
movies_df = pd.read_csv(r"C:\Users\Linzhiping\Jupyter\ml-latest-small\movies.csv")

movies_df['movieRow']= movies_df.index  #添加电影行数


        userId  movieId  rating   timestamp
100831     610   166534     4.0  1493848402
100832     610   168248     5.0  1493850091
100833     610   168250     5.0  1494273047
100834     610   168252     5.0  1493846352
100835     610   170875     3.0  1493846415


In [16]:
ratings_df.tail()

,userId,movieId,rating,timestamp
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352
100835,610,170875,3.0,1493846415


In [17]:
movies_df.tail()

,movieId,title,genres,movieRow
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,9737
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,9738
9739,193585,Flint (2017),Drama,9739
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,9740
9741,193609,Andrew Dice Clay: Dice Rules (1991),Comedy,9741


In [19]:
movies_df = movies_df[["movieRow","movieId","title"]]
movies_df.to_csv('moviesProcessed.csv',index = False,header = True,encoding ="utf-8")
ratings_df =pd.merge(ratings_df,movies_df,on = 'movieId')

In [21]:
ratings_df = ratings_df[["userId","movieRow","rating"]]
ratings_df.to_csv('ratingsProcessed.csv',index =False,header =True,encoding ='utf-8')

In [23]:
userNo =ratings_df['userId'].max()+1
movieNo =movies_df['movieRow'].max()+1
rating = np.zeros((movieNo,userNo))
flag =0
ratings_df_length = np.shape(ratings_df)[0]
for index,row in ratings_df.iterrows():
    rating[int(row["movieRow"]),int(row["userId"])] = row["rating"]
    flag +=1
#    print ( "processed %d,%d left" %(flag,ratings_df_length-flag))

In [27]:
record = rating > 0
record =np.array(record,dtype=int)

In [28]:
record

array([[0, 1, 0, ..., 1, 1, 1],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 1, 0, ..., 1, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [34]:
def normalizeRatings(rating,record):
    m,n = rating.shape  #m,n分别为电影数，用户数
    rating_mean = np.zeros((m,1)) #记录每部电影的平均分
    rating_norm = np.zeros((m,n)) #正规化矩阵
    for i in range(m):
        idx = record[i,:] != 0   #记录那些评分不为零的下标
        rating_mean[i] = np.mean(rating[i,idx]) #求平均
        rating_norm [i,idx] -= rating_mean[i] #求正规，好像有负数
    return rating_norm,rating_mean

In [64]:
rating_norm,rating_mean = normalizeRatings(rating,record)
rating_norm =np.nan_to_num(rating_norm)
rating_mean =np.nan_to_num(rating_mean)
num_features=10
X_parameters = tf.Variable(tf.random_normal([movieNo,num_features],stddev = 0.35))
Theta_parameters =tf.Variable(tf.random_normal([userNo,num_features],stddev=0.35))
#构建损失函数loss
loss = 1/2 * tf.reduce_sum(((tf.matmul(X_parameters,Theta_parameters,transpose_b = True)- rating_norm)*record)**2)+ 1/2 *(tf.reduce_sum(X_parameters**2) + tf.reduce_sum(Theta_parameters**2))

#将X_parameters，Theta_parameters矩阵相乘相乘之前将Theta_parameters转置
#创建优化器和优化目标
optimizer = tf.train.AdamOptimizer(1e-2)
train = optimizer.minimize(loss)

In [65]:
tf.summary.scalar('loss',loss)
summaryMerged = tf.summary.merge_all()
filename = 'movie_tensorboard'
writer = tf.summary.FileWriter(filename)
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

In [66]:
for i in range(900):
    _,movie_summary = sess.run([train,summaryMerged])
    writer.add_summary(movie_summary, i)

In [68]:
Current_X_parameters,Current_Theta_parameters = sess.run([X_parameters,Theta_parameters])
predicts = np.dot(Current_X_parameters,Current_Theta_parameters.T)+rating_mean
errors = np.sqrt(np.sum((predicts-rating)**2))


In [69]:
errors

2832.064072065368

In [70]:
user_id = input('请输入要推荐的用户编号：')
sortedResult = predicts[:,int(user_id)].argsort()[::-1]#获取该用户的电影评分列表
idx =0
print('为该用户提供的推荐列表：'.center(80,'='))
for i in sortedResult:
    print('评分：%.2f,电影名：%s'%(predicts[i,int(user_id)],movies_df.iloc[i]['title']))
    idx +=1
    if idx == 20:break


请输入要推荐的用户编号：213
==================================为该用户提供的推荐列表：==================================
评分：4.14,电影名：Girls About Town (1931)
评分：4.14,电影名：Little Murders (1971)
评分：4.13,电影名：Tom and Jerry: Shiver Me Whiskers (2006)
评分：4.13,电影名：My Life as McDull (Mak dau goo si) (2001)
评分：4.13,电影名：World of Glory (1991)
评分：4.13,电影名：Scooby-Doo! Curse of the Lake Monster (2010)
评分：4.13,电影名：Bloodsucking Bastards (2015)
评分：4.13,电影名：Happy Feet Two (2011)
评分：4.13,电影名：Scooby-Doo! and the Loch Ness Monster (2004)
评分：4.13,电影名：My Left Eye Sees Ghosts (Ngo joh aan gin diy gwai) (2002)
评分：4.13,电影名：The Love Bug (1997)
评分：4.13,电影名：Ice Age: The Great Egg-Scapade (2016)
评分：4.13,电影名：Saving Santa (2013)
评分：4.13,电影名：Watching the Detectives (2007)
评分：4.13,电影名：The Eye: Infinity (2005)
评分：4.13,电影名：Go for Zucker! (Alles auf Zucker!) (2004)
评分：4.13,电影名：National Lampoon's Bag Boy (2007)
评分：4.13,电影名：Scooby-Doo! and the Samurai Sword (2009)
评分：4.13,电影名：Hunting Elephants (2013)
评分：4.13,电影名：Breed, The (2006)
